In [1]:
import sys
sys.path.insert(1, '/home/b3arjuden/crocket')

from decimal import Decimal
from json import load as json_load
from os import environ
from os.path import join

from crocket.sql.sql import Database
from crocket.utilities.passcode import AESCipher
from crocket.utilities.credentials import get_credentials

In [2]:
# Environment variables

HOME_DIRECTORY_PATH = environ['HOME']
CREDENTIALS_FILE_PATH = join(HOME_DIRECTORY_PATH, '.credentials_unlocked.json')

HOSTNAME = 'localhost'
TRADE_DATABASE = 'TRADEBOT_RECORD'
MARKET_DATABASE = 'PRODUCTION10'

In [3]:
# Load SQL username and password
USERNAME, PASSCODE = get_credentials(CREDENTIALS_FILE_PATH)

In [4]:
trade_db = Database(hostname=HOSTNAME,
                    username=USERNAME,
                    password=PASSCODE,
                    database_name=TRADE_DATABASE)

data_db = Database(hostname=HOSTNAME,
                   username=USERNAME,
                   password=PASSCODE,
                   database_name=MARKET_DATABASE)

In [5]:
trades = trade_db.select_query(MARKET_DATABASE, '*')

In [7]:
market = data_db.select_query('BTC-OMG', '*')

In [24]:
trades

[(1,
  'BTC-OMG',
  datetime.datetime(2017, 12, 8, 3, 3, 13),
  Decimal('0.00055554'),
  Decimal('0.00200543'),
  datetime.datetime(2017, 12, 8, 3, 6, 23),
  Decimal('0.00054895'),
  Decimal('0.00197176'),
  Decimal('-0.00003367'),
  Decimal('-1.6789')),
 (2,
  'BTC-ETC',
  datetime.datetime(2017, 12, 8, 3, 3, 4),
  Decimal('0.00165139'),
  Decimal('0.00200911'),
  datetime.datetime(2017, 12, 8, 3, 29, 17),
  Decimal('0.00173799'),
  Decimal('0.00210392'),
  Decimal('0.00009481'),
  Decimal('4.7190')),
 (3,
  'BTC-BTG',
  datetime.datetime(2017, 12, 8, 3, 17, 52),
  Decimal('0.01656472'),
  Decimal('0.00203390'),
  datetime.datetime(2017, 12, 8, 3, 29, 37),
  Decimal('0.01687882'),
  Decimal('0.00206214'),
  Decimal('0.00002824'),
  Decimal('1.3885')),
 (4,
  'BTC-STRAT',
  datetime.datetime(2017, 12, 8, 3, 13, 1),
  Decimal('0.00054986'),
  Decimal('0.00200565'),
  datetime.datetime(2017, 12, 8, 3, 33, 9),
  Decimal('0.00053842'),
  Decimal('0.00195414'),
  Decimal('-0.00005151'),
  D

In [8]:
market

[(datetime.datetime(2017, 12, 8, 1, 44, 41),
  Decimal('0.00053160'),
  Decimal('0.00053061'),
  Decimal('0.63541407'),
  Decimal('0.55603873'),
  Decimal('0.07937534'),
  21,
  3),
 (datetime.datetime(2017, 12, 8, 1, 45, 41),
  Decimal('0.00054099'),
  Decimal('0.00054220'),
  Decimal('2.54080166'),
  Decimal('1.92611258'),
  Decimal('0.61468908'),
  37,
  19),
 (datetime.datetime(2017, 12, 8, 1, 46, 41),
  Decimal('0.00053785'),
  Decimal('0.00053707'),
  Decimal('0.26452023'),
  Decimal('0.08649904'),
  Decimal('0.17802119'),
  5,
  10),
 (datetime.datetime(2017, 12, 8, 1, 47, 41),
  Decimal('0.00053712'),
  Decimal('0.00053646'),
  Decimal('0.33890935'),
  Decimal('0.01067296'),
  Decimal('0.32823639'),
  5,
  16),
 (datetime.datetime(2017, 12, 8, 1, 48, 41),
  Decimal('0.00053840'),
  Decimal('0.00053925'),
  Decimal('0.07248416'),
  Decimal('0.05807818'),
  Decimal('0.01440598'),
  4,
  3),
 (datetime.datetime(2017, 12, 8, 1, 49, 41),
  Decimal('0.00053929'),
  Decimal('0.0005399

In [14]:
from time import strftime
from datetime import timedelta

In [11]:
trades[0][1]

'BTC-OMG'

In [15]:
(trades[0][2] - timedelta(minutes=1)).strftime("%Y-%m-%d %H:%M:%S")

'2017-12-08 03:02:13'

In [26]:
buy_diff = []
sell_diff = []

for t in trades:
    
    buy_time = t[2]
    buy_time_str = (buy_time - timedelta(minutes=1)).strftime("%Y-%m-%d %H:%M:%S")
    
    sell_time = t[5]
    sell_time_str = (sell_time - timedelta(minutes=1)).strftime("%Y-%m-%d %H:%M:%S")
    
    market = data_db.select_query(t[1], '*', 'where time > "{}" limit 420'.format(buy_time_str))
    
    for index in range(len(market)-1):
        
        time_now = market[index][0]
        time_next = market[index+1][0]
        
        if time_now < buy_time < time_next:
            
            diff = (t[3] - market[index][2]) / market[index][2] * 100
            print(t[1], 'BUY DIFF: {}'.format(str(diff)))
            buy_diff.append(diff)
            
        elif time_now < sell_time < time_next:
    
            diff = (t[6] - market[index][2]) / market[index][2] * 100
            print(t[1], 'SELL DIFF: {}'.format(str(diff)))
            sell_diff.append(diff)
            break

BTC-OMG BUY DIFF: 0.07205389631444320351623014014
BTC-OMG SELL DIFF: -0.2634447674418604651162790698
BTC-ETC BUY DIFF: 0.3323369301059589773500534655
BTC-ETC SELL DIFF: -0.01495757227096217460089170142
BTC-BTG BUY DIFF: 0.9950297289519507948043839839
BTC-BTG SELL DIFF: -0.2446776696756897959955603414
BTC-STRAT BUY DIFF: 0.1402320202516891584257589830
BTC-STRAT SELL DIFF: -0.3829858091731576902440378175
BTC-XLM BUY DIFF: -0.3504672897196261682242990654
BTC-XLM SELL DIFF: -0.3500583430571761960326721120
BTC-XMR BUY DIFF: 0.06700455689506150082014748025
BTC-XMR SELL DIFF: -0.0002373432273813387700755285485
BTC-SBD BUY DIFF: 1.192157416560749423339452996
BTC-SBD SELL DIFF: -1.601931095018652622339258284
BTC-IOP BUY DIFF: 0.6098310669584675535446430767
BTC-IOP SELL DIFF: 1.548580660050773136395107316
BTC-BTG BUY DIFF: 0.01146436170385027673155581752
BTC-BTG SELL DIFF: -1.928315582472006199372486882
BTC-VTC BUY DIFF: 0.4134753588375435625824489555
BTC-VTC SELL DIFF: 0.75459580958497230472826